In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch

2024-03-21 21:06:32.800528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 21:06:33.276887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BATCH_SIZE = 256

In [3]:
AS_dataset = pd.read_csv('./../Arbitrary_Single_band_Coupler_Phase_Shift.csv', encoding='utf-8').sample(frac=1).reset_index(drop=True)
full_X = AS_dataset.loc[:,'freq':'L4'].to_numpy(dtype = np.float32)
full_y = AS_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
X_train, X_vali, y_train, y_vali = train_test_split(full_X, full_y, test_size=0.05, random_state=0)

In [4]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.shuffle(buffer_size=X_train.shape[0])
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2024-03-21 21:06:35.074699: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-03-21 21:06:35.075259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [5]:
dataset_vali = tf.data.Dataset.from_tensor_slices((X_vali, y_vali))
dataset_vali = dataset_vali.shuffle(buffer_size=X_vali.shape[0])
dataset_vali = dataset_vali.batch(BATCH_SIZE)
dataset_vali = dataset_vali.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
# 定义模型构建函数
def build_model(hp):
    model = tf.keras.Sequential()
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=1024, step=32), activation='leaky_relu'))
        if hp.Boolean('use_batchnorm' + str(i)):
            model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(8))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mse')
    return model

In [7]:
# 创建 tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=1024, # 根据实际情况调整试验次数
    directory='my_dir',
    project_name='mlp_project1')

Reloading Tuner from my_dir/mlp_project1/tuner0.json


In [ ]:
# 开始搜索过程
tuner.search(dataset_train,
             epochs=50,
            validation_data=dataset_vali)

Trial 991 Complete [00h 01m 29s]
val_loss: 0.012802664190530777

Best val_loss So Far: 0.004853558726608753
Total elapsed time: 19h 35m 05s

Search: Running Trial #992

Value             |Best Value So Far |Hyperparameter
3                 |4                 |num_layers
736               |256               |units_0
False             |True              |use_batchnorm0
0.001             |0.001             |learning_rate
896               |704               |units_1
True              |False             |use_batchnorm1
32                |928               |units_2
True              |True              |use_batchnorm2
992               |640               |units_3
True              |False             |use_batchnorm3
128               |416               |units_4
False             |True              |use_batchnorm4

Epoch 1/50
334/334 [==============================] - 3s 5ms/step - loss: 0.0965 - val_loss: 0.0817
Epoch 2/50
334/334 [==============================] - 2s 5ms/step - loss: 0.0618 

In [ ]:
# 获取最优模型
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tuner.results_summary()